<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Baixando pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make -j4 

In [2]:
#@title Importando bibliotecas {display-mode: "form"}

from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyConformer import *
from pyRAND import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *
from pyMC import *
from pyFullSearch import *
from pyDocker import *

In [1]:
#@title Baixando base de dados proteína ligante {display-mode: "form"}

from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/pyLiBELa/SB/'

with open(path+'list.txt') as f:
    pdb_codes_list = f.readlines()

pdb_codes_list = [pdb_code[0:4] for pdb_code in pdb_codes_list]

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.x_dim = 5.0 #@param {type:"number"}
Input.y_dim = 5.0 #@param {type:"number"}
Input.z_dim = 5.0 #@param {type:"number"}
Input.write_grids = True;

In [4]:
import os
pdb_code = pdb_codes_list[0]
for pdb_code in pdb_codes_list[0:2]:
  print(pdb_code)


  Input = PARSER()
  Writer = WRITER(Input)
  Coord = COORD_MC()
  HB = FindHB()
  Input.grid_prefix = "McGrid_"+pdb_code

  lig_src =  path + pdb_code +'/' + pdb_code + '.lig.am1bcc.mol2.gz'
  rec_src = path + pdb_code +'/' + pdb_code + '.rec.clean.mol2.gz'

  lig = Mol2(Input, lig_src)
  rec = Mol2(Input, rec_src)
  ref_lig = Mol2(Input, lig_src)

  for i in range(len(list(rec.residue_pointer))-1):
    HB.parse_residue(rec.residue_pointer[i]-1, rec.residue_pointer[i+1]-2, rec.resnames[i], rec, lig, 9.0)

  HB.find_ligandHB(lig_src, lig)

  com = Coord.compute_com(lig)

  %time Grids = Grid(Input, Writer, rec, com)

  Energy = Energy2(Input)
  energy1 = Energy.compute_ene(rec, lig, lig.xyz)
  print("Original energy: %7.3f" % energy1)

  energy2 = Energy.compute_ene(Grids, lig, lig.xyz)
  print("Grid energy: %7.3f" % energy2)
  

#  !mv *.grid $folder

121P
CPU times: user 26.7 s, sys: 189 ms, total: 26.8 s
Wall time: 27 s
Original energy: -237.910
Grid energy: 2999795.409
181L
CPU times: user 24.9 s, sys: 134 ms, total: 25 s
Wall time: 25.3 s
Original energy: -12.322
Grid energy: -12.179


In [39]:
for pdb_code in pdb_codes_list[0:4]:
  folder = path+pdb_code+'/'
  !ls $folder

121P.lig.am1bcc.mol2.gz  121P.rec.clean.mol2.gz  McGrid_121P.grid  McLiBELa.log
181L.lig.am1bcc.mol2.gz  181L.rec.clean.mol2.gz  McLiBELa.log
182L.lig.am1bcc.mol2.gz  182L.rec.clean.mol2.gz  McLiBELa.log
183L.lig.am1bcc.mol2.gz  183L.rec.clean.mol2.gz  McLiBELa.log
